In [ ]:
# importing libs
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda,Dense, GaussianNoise
from keras.models import Model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD
import random as rn

In [ ]:
# defining parameters
M = 2
R = 1/4
k = int(np.log2(M))
n_channel = int(k/R)

print ('M:',M,'k:',k,'n_coded:',n_channel,'R:',R)

M: 2 k: 1 n_coded: 4 R: 0.25


In [ ]:
#generating data of size N
N = 1000000
label = np.random.randint(M,size=N)

In [ ]:
# creating one hot encoded vectors
data = []
for i in label:
    temp = np.zeros(M)
    temp[i] = 1
    data.append(temp)

In [ ]:
data = np.array(data)
print (data.shape)

(1000000, 2)


In [ ]:
temp_check = [17,23,45,67,89,96,72,250,350]
for i in temp_check:
    print(label[i],data[i])

1 [0. 1.]
1 [0. 1.]
1 [0. 1.]
0 [1. 0.]
1 [0. 1.]
1 [0. 1.]
0 [1. 0.]
0 [1. 0.]
0 [1. 0.]


In [ ]:
lr=0.001
Eb_N0db=5
Eb_N0=10.0**(Eb_N0db/10.0)

In [ ]:
# defining autoencoder and it's layer
input_signal = Input(shape=(M,))
encoded = Dense(M, activation='relu')(input_signal)
encoded1 = Dense(M, activation='relu')(encoded)
encoded2 = Dense(2*n_channel, activation='linear')(encoded1)
encoded3 = Lambda(lambda x: np.sqrt(n_channel)*K.l2_normalize(x,axis=1))(encoded2)

encoded4 = GaussianNoise(np.sqrt(1/(2*R*Eb_N0)))(encoded3)

decoded = Dense(M, activation='relu')(encoded4)
decoded1 = Dense(M, activation='relu')(decoded)
decoded2 = Dense(M, activation='softmax')(decoded1)

autoencoder = Model(input_signal, decoded2)
adam = Adam(lr)
autoencoder.compile(optimizer=adam, loss='categorical_crossentropy')

In [ ]:
print (autoencoder.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 24        
_________________________________________________________________
lambda (Lambda)              (None, 8)                 0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 18    

In [ ]:
autoencoder.fit(data, data,
                epochs=50,
                batch_size=2000)

Epoch 1/50
500/500 [==============================] - 2s 3ms/step - loss: 0.4451
Epoch 2/50
500/500 [==============================] - 1s 3ms/step - loss: 0.1478
Epoch 3/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0838
Epoch 4/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0557
Epoch 5/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0416
Epoch 6/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0340
Epoch 7/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0294
Epoch 8/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0258
Epoch 9/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0233
Epoch 10/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0213
Epoch 11/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0206
Epoch 12/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0195
Epoch 13/50
500/500 [====

In [ ]:
from keras.models import load_model
#autoencoder.save('4_7_symbol_autoencoder_v_best.model')

In [ ]:
#autoencoder_loaded = load_model('4_7_symbol_autoencoder_v_best.model')

In [ ]:
encoder = Model(input_signal, encoded3)

In [ ]:
# making decoder from full autoencoder
encoded_input = Input(shape=(2*n_channel,))

deco = autoencoder.layers[-3](encoded_input)
deco1 = autoencoder.layers[-2](deco)
deco2 = autoencoder.layers[-1](deco1)
decoder = Model(encoded_input, deco2)

In [ ]:
N = 1000000
test_label = np.random.randint(M,size=N)
test_data = []

for i in test_label:
    temp = np.zeros(M)
    temp[i] = 1
    test_data.append(temp)
    
test_data = np.array(test_data)

In [ ]:
temp_test = 6
print (test_data[temp_test][test_label[temp_test]],test_label[temp_test])

1.0 0


In [ ]:
a=encoder.predict(test_data)
noi = np.sqrt(1/(2*R)) * np.random.randn(N,2*n_channel)
b=a + noi
pred=decoder.predict(b)

In [ ]:
(pred.round()!=test_data).sum()/8000000

0.010163125

In [ ]:
test_label

array([ 8,  5,  1, ...,  4,  3, 12])

In [ ]:
(pred.round()!=test_data).mean()

0.0050815625

In [ ]:
def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

# BER

Mean

In [ ]:
EbNodB_range = list(frange(0,11,1))
ber = [None]*len(EbNodB_range)
for i in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[i]/10.0)
    noise_std = np.sqrt(1/(2*R*EbNo))
    noise_mean = 0
    no_errors = 0
    noise = noise_std * np.random.randn(N,2*n_channel)
    encoded_signal = encoder.predict(test_data) 
    final_signal = encoded_signal + noise
    pred_final_signal =  decoder.predict(final_signal)
    ber[i] = (pred_final_signal.round()!=test_data).mean()
    print ('SNR:',EbNodB_range[i],'BER:',ber[i])

SNR: 0 BER: 0.078787
SNR: 1 BER: 0.056281
SNR: 2 BER: 0.037739
SNR: 3 BER: 0.023102
SNR: 4 BER: 0.01251
SNR: 5 BER: 0.006075
SNR: 6 BER: 0.002299
SNR: 7 BER: 0.000761
SNR: 8 BER: 0.000166
SNR: 9 BER: 4.9e-05
SNR: 10 BER: 5e-06


### Matlab Array form

In [ ]:
for n in range(0,len(EbNodB_range)):
  print(ber[n], " ",end='')

0.078787  0.056281  0.037739  0.023102  0.01251  0.006075  0.002299  0.000761  0.000166  4.9e-05  5e-06  